# Лабораторная работа №2

In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Загружаем датасет со свойствами различных грибов, целевым параметром является съедобность (значение "e" целевого параметра) или несъедобность ("p") гриба.

In [2]:
mushrooom_dataset = pd.read_csv('MushroomDataset/secondary_data.csv',delimiter=';')

Выделяем таргетный столбец в отдельный Series, а все остальные столбы в DataFrame. Также удаляем все столбцы, где большая часть данных типа NaN и заменяем все классы на номер буквы в алфавите.

In [20]:
sample_size=50000
y = mushrooom_dataset['class'].sample(n=sample_size)
X = mushrooom_dataset.drop('class',axis=1).sample(n=sample_size)
X = X.drop(columns=['cap-surface','gill-spacing','gill-attachment','stem-root',
                    'stem-surface','veil-type','veil-color','ring-type','spore-print-color'])
X.replace({'a':1,'b':2,'c':3,'d':4,'e':5,'f':6,'g':7,
           'h':8,'i':9,'j':10,'k':11,'l':12,'m':13,
           'n':14,'o':15,'p':16,'q':17,'r':18,'s':19,
           't':20,'u':21,'v':22,'w':23,'x':24,'y':25,'z':26},inplace=True)

После устранения пропусков нормализуем все значения.

In [21]:
scaler = StandardScaler()
X_ans = scaler.fit_transform(X)

Разделяем данные на тренировочную и тестовую части.

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.2)

Найдём лучшие гиперпараметры, для этого используем GridSearchCV, куда передадим диапазон возможных значений. Будем искать оптимальное значение гиперпараметров C, gamma и kernel, которые отвечают за силу поправок, коэффициент для ядер 'rbf' и 'sigmoid' и используемое ядро.

In [23]:
classifier = SVC()
params ={
    'C': [1,0.1,0.01,0.001],
    'gamma': [1,10,100,1000],
    'kernel':['linear','sigmoid','rbf']
}
grid = GridSearchCV(classifier, params, cv=2, n_jobs=-1,verbose=3)
grid.fit(X_train, y_train)
best_params = grid.best_params_

Fitting 2 folds for each of 48 candidates, totalling 96 fits


После того, как были найдены оптимальные гиперпараметры, можно вывести различные метрики, такие как точность, полнота, f-мера и поддержка.

In [24]:
best_model = SVC(**best_params)
best_model.fit(X_train, y_train)
predicted = best_model.predict(X_test)
print('Used params:',best_params)
print('Evaluation:\n', metrics.classification_report(y_test, predicted,digits=5))

Used params: {'C': 1, 'gamma': 1, 'kernel': 'linear'}
Evaluation:
               precision    recall  f1-score   support

           e    0.00000   0.00000   0.00000      4507
           p    0.54930   1.00000   0.70909      5493

    accuracy                        0.54930     10000
   macro avg    0.27465   0.50000   0.35455     10000
weighted avg    0.30173   0.54930   0.38951     10000



c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
